## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Smithers,CA,2021-12-21 19:0745,54.7804,-127.1743,7.12,85,90,0.00,light snow
1,1,Kapaa,US,2021-12-21 19:0746,22.0752,-159.3190,73.72,77,90,4.00,light rain
2,2,Warri,NG,2021-12-21 19:0746,5.5167,5.7500,79.68,51,3,6.40,clear sky
3,3,Erenhot,CN,2021-12-21 19:0746,43.6475,111.9767,18.16,62,0,7.56,clear sky
4,4,Saint-Paul,RE,2021-12-21 19:0747,-21.0096,55.2707,78.91,96,100,2.13,light rain


In [26]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?85


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#Filter the data set to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Warri,NG,2021-12-21 19:0746,5.5167,5.7500,79.68,51,3,6.40,clear sky
4,4,Saint-Paul,RE,2021-12-21 19:0747,-21.0096,55.2707,78.91,96,100,2.13,light rain
5,5,Atuona,PF,2021-12-21 19:0747,-9.8000,-139.0333,78.62,73,100,17.76,overcast clouds
6,6,Ancud,CL,2021-12-21 19:0747,-41.8697,-73.8203,75.22,64,0,7.72,clear sky
7,7,Jutai,BR,2021-12-21 19:0748,-5.1833,-68.9000,84.65,83,89,2.37,overcast clouds
14,14,Wajir,KE,2021-12-21 19:0749,1.7471,40.0573,79.93,72,100,12.55,light rain
18,18,Itarema,BR,2021-12-21 19:0750,-2.9248,-39.9167,84.24,64,100,14.90,overcast clouds
20,20,Cabo San Lucas,MX,2021-12-21 19:0356,22.8909,-109.9124,80.62,57,1,5.99,clear sky
21,21,Hambantota,LK,2021-12-21 19:0751,6.1241,81.1185,77.34,100,0,2.30,clear sky
25,25,Alofi,NU,2021-12-21 19:0752,-19.0595,-169.9187,80.49,74,75,14.97,broken clouds


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Warri,NG,2021-12-21 19:0746,5.5167,5.7500,79.68,51,3,6.40,clear sky
4,4,Saint-Paul,RE,2021-12-21 19:0747,-21.0096,55.2707,78.91,96,100,2.13,light rain
5,5,Atuona,PF,2021-12-21 19:0747,-9.8000,-139.0333,78.62,73,100,17.76,overcast clouds
6,6,Ancud,CL,2021-12-21 19:0747,-41.8697,-73.8203,75.22,64,0,7.72,clear sky
7,7,Jutai,BR,2021-12-21 19:0748,-5.1833,-68.9000,84.65,83,89,2.37,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...
682,682,Cozumel,MX,2021-12-21 19:2443,20.5083,-86.9458,82.47,73,20,13.80,few clouds
686,686,Mogadishu,SO,2021-12-21 19:2444,2.0371,45.3438,79.18,82,100,20.69,overcast clouds
688,688,Piacabucu,BR,2021-12-21 19:2444,-10.4056,-36.4344,83.97,66,51,15.32,broken clouds
689,689,Grand Gaube,MU,2021-12-21 19:2445,-20.0064,57.6608,79.20,91,100,5.99,light rain


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Warri,NG,79.68,clear sky,5.5167,5.7500,
4,Saint-Paul,RE,78.91,light rain,-21.0096,55.2707,
5,Atuona,PF,78.62,overcast clouds,-9.8000,-139.0333,
6,Ancud,CL,75.22,clear sky,-41.8697,-73.8203,
7,Jutai,BR,84.65,overcast clouds,-5.1833,-68.9000,
14,Wajir,KE,79.93,light rain,1.7471,40.0573,
18,Itarema,BR,84.24,overcast clouds,-2.9248,-39.9167,
20,Cabo San Lucas,MX,80.62,clear sky,22.8909,-109.9124,
21,Hambantota,LK,77.34,clear sky,6.1241,81.1185,
25,Alofi,NU,80.49,broken clouds,-19.0595,-169.9187,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    hotel_loc = f"{lat},{lng}"
    params['location']= hotel_loc
    
    #Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,  params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:

# 7. Drop the rows where there is no Hotel Name.
null_hotels = hotel_df.loc[hotel_df["Hotel Name"]==""]
hotels_to_drop = null_hotels.index
clean_hotel_df = hotel_df.drop(index = hotels_to_drop)
clean_hotel_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 2 to 692
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 158 non-null    object 
 1   Country              158 non-null    object 
 2   Max Temp             158 non-null    float64
 3   Weather Description  158 non-null    object 
 4   Lat                  158 non-null    float64
 5   Lng                  158 non-null    float64
 6   Hotel Name           158 non-null    object 
dtypes: float64(3), object(4)
memory usage: 9.9+ KB


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPY_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 2 to 692
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 158 non-null    object 
 1   Country              158 non-null    object 
 2   Max Temp             158 non-null    float64
 3   Weather Description  158 non-null    object 
 4   Lat                  158 non-null    float64
 5   Lng                  158 non-null    float64
 6   Hotel Name           158 non-null    object 
dtypes: float64(3), object(4)
memory usage: 9.9+ KB


In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 2 to 692
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     158 non-null    float64
 1   Lng     158 non-null    float64
dtypes: float64(2)
memory usage: 3.7 KB


In [35]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))